In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

In [8]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False 
opciones.add_argument('--start-maximized')
opciones.add_argument('--incognito')     

In [3]:
data = pd.read_csv("../mydata/data.csv")
data.index=data.date
data.drop('date',axis=1,inplace=True)

In [9]:
e=['ajo','albaricoque']

In [56]:
driver = "../driver/chromedriver.exe"
driver = webdriver.Chrome(driver, options=opciones)
supermarket = "carrefour"
prices = []
prod = []
sup = []
links = []
names = []
frame = {}
for f in e:
    for i in range(2,4):
        url = f"https://www.carrefour.es/?q={f}&page={i}"
        driver.get(url)
        time.sleep(0.3)
        try:
            driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
            time.sleep(0.7)
        except:
            pass
        try:
            for o in range(1,50):
                    prices.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > span').text.split("€")[0])
                    driver.execute_script("window.scrollTo(0, window.scrollY + 300)")
                    prod.append(f)
                    sup.append('carrefour')
                    links.append(url)
                    names.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > a.ebx-result-link.ebx-result__title-link.track-click > h1').text)
        except:
            pass

frame['prices'] = prices
frame['product'] = prod
frame['supermarket'] = sup
frame['links'] = links
frame['names'] = names

scrap = pd.DataFrame(frame)
scrap.dropna(how='any',inplace=True)
if not os.path.exists(f"../mydata/scraps_{supermarket}"):
    os.makedirs(f"../mydata/scraps_{supermarket}")
    now = str(datetime.now())[:19].replace(":","_")
    scrap.to_csv(f'../mydata/scraps_{supermarket}/{now}.csv')

/tmp/ipykernel_429/2705862167.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver, options=opciones)
/tmp/ipykernel_429/2705862167.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
/tmp/ipykernel_429/2705862167.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  prices.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > span').text.split("€")[0])
/tmp/ipykernel_429/2705862167.py:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  names.append(driver.find_element_by_css_selector(f'#ebx-grid > article:nth-child({o}) > div > a.ebx-result-link.ebx-result__title-link.track-click > h1').text)
